In [ ]:
df = df.sort_values(by='계약일자')

In [ ]:
from tqdm import tqdm
from collections import Counter

## 먼저 각 고객별로 구매량이 얼마인가, 고객 고유치 : 51397
top_data = Counter(df['고객ID']).most_common() ## 데이터 개수가 많은 순으로 출력
monetary_df = pd.DataFrame() ## 구매량 데이터 초기화
# monetary_df['고객ID'] = top_data ## 고객아이디 삽입

data = [x[1] for x in top_data] ## 데이터 개수
cus = [x[0] for x in top_data] ## 고객

customer_id = list(cus)
monetary_df['고객ID'] = customer_id

monetary_data = [] ## 구매량을 담을 리스트
for i in tqdm(data, position=0, desc='Calculating amount of individual customer'):
    monetary_data.append(i) ## 해당 고객 구매량

monetary_df['Monetary'] = monetary_data ## 구매량 데이터 삽입
monetary_df['고객ID'] = cus

## 각 고객별 최근방문일을 알아보자.
temp_recency_df = df[['고객ID','계약일자']].drop_duplicates() ## 고객 아이디와 계약일자만 추출한뒤 중복 제거
recency_df = temp_recency_df.groupby('고객ID')['계약일자'].max().reset_index() ## 아이디로 그룹화 한다음 최근방문일을 구해야하므로 날짜에 max를 적용한다.
recency_df = recency_df.rename(columns={'계약일자':'Recency'})

## 각 고객별 방문횟수를 알아보자.
temp_frequency_df = df[['고객ID','계약일자']].drop_duplicates() ## 고객 아이디와 계약일자만 추출한뒤 중복 제거
frequency_df = temp_frequency_df.groupby('고객ID')['계약일자'].count().reset_index() ## 아이디로 그룹화 한다음 방문횟수를 구해야 한다. 여기서는 방문횟수를 계약일수로 생각했으므로 계약일수에 count를 적용한다.
frequency_df = frequency_df.rename(columns={'계약일자':'Frequency'})

## 데이터를 고객아이디를 기준으로 합쳐야한다.
rfm_df = pd.merge(recency_df,frequency_df,how='left',on='고객ID')
rfm_df = pd.merge(rfm_df,monetary_df,how='left',on='고객ID')

Calculating amount of individual customer: 100%|██████████| 32492/32492 [00:00<00:00, 2232326.91it/s]


In [ ]:
monetary_df.nunique()

고객ID        32492
Monetary       66
dtype: int64

In [ ]:
rfm_df.nunique()

고객ID         32492
Recency       3491
Frequency       19
Monetary        66
dtype: int64

In [ ]:
rfm_df

,고객ID,Recency,Frequency,Monetary
0,A0063H17IS0OELIO,2018-04-06,1,1
1,A007E35OUXTJ3BRR,2021-06-22,1,1
2,A008UGCQHSONWYDJ,2019-09-25,1,1
3,A00AXO66OKGQAC8F,2021-09-14,1,1
4,A00BD03LKJIH6LTU,2016-11-02,2,4
...,...,...,...,...
32487,AZZY0IZN75VOACLI,2018-12-03,1,1
32488,AZZYI8XJU7OCXO73,2011-06-27,1,1
32489,AZZYZ8SXW66M0RBL,2021-09-15,2,2
32490,AZZZ70FWYOY16G62,2011-02-18,3,6


In [ ]:
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 5.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 7.8 MB/s eta 0:00:0000:0100:01


In [ ]:
from sklearn import preprocessing

def get_score(level, data):

    score = [] 
    for j in range(len(data)): 
        for i in range(len(level)): 
            if data[j] <= level[i]: 
                score.append(i+1) 
                break 
            elif data[j] > max(level): 
                score.append(len(level)+1) 
                break 
            else: 
                continue
    return score
    
def get_rfm_grade(df, num_class, rfm_tick_point, rfm_col_map, suffix=None):
##### 최근방문일/방문횟수/구매금액 점수 부여
    for k, v in rfm_tick_point.items():
        if isinstance(v,str):
            if v == 'quantile':
                ## 데이터 변환
                scale = preprocessing.StandardScaler() ## 데이터의 범위 조작하기 쉽게 해주는 클래스 
                temp_data = np.array(df[rfm_col_map[k]]) ## 데이터를 Numpy 배열로 변환
                temp_data = temp_data.reshape((-1,1)) ## scale을 적용하기위해 1차원 배열을 2차원으로 변환
                temp_data = scale.fit_transform(temp_data) ## 데이터를 평균은 0, 표준편차는 1을 갖도록 변환 
                temp_data = temp_data.squeeze() ## 데이터를 다시 1차원으로 변환
 
                ## 분위수 벡터
                quantiles_level = np.linspace(0,1,num_class+1)[1:-1] ## 분위수를 구할 기준값을 지정 0과 1은 제외
                quantiles = [] ## 분위수를 담을 리스트
                for ql in quantiles_level:
                    quantiles.append(np.quantile(temp_data,ql)) ## 분위수를 계산하고 리스트에 삽입
            else: ## min_max인 경우
                ## 데이터 변환
                temp_data = np.array(df[rfm_col_map[k]])
 
                ## 등분점 계산
                quantiles = np.linspace(np.min(temp_data),np.max(temp_data),num_class+1)[1:-1] ## 최소값과 최대값을 점수 개수만큼 등간격으로 분할하는 점
        else: ## 직접 구분값을 넣어주는 경우
            temp_data = np.array(df[rfm_col_map[k]])
            quantiles = v ## 직접 구분값을 넣어줌
        score = get_score(quantiles, temp_data) ## 구분값을 기준으로 점수를 부여하고 리스트로 저장한다.
        new_col_name = rfm_col_map[k]+'_'+k ## 점수값을 담는 변수의 이름
        if suffix:
            new_col_name = rfm_col_map[k]+'_'+suffix
        df[new_col_name] = score ## 기존데이터 옆에 점수 데이터를 추가한다.
    return df

In [ ]:
##분위수를 이용해 점수 부여

rfm_tick_point={'R':'quantile','F':'quantile','M':'quantile'}
rfm_col_map={'R':'Recency','F':'Frequency','M':'Monetary'}

result = get_rfm_grade(df=rfm_df, num_class=5, rfm_tick_point=rfm_tick_point, rfm_col_map=rfm_col_map)
result.to_csv('result.csv',index=False)

In [ ]:
result[['고객ID','Recency_R','Frequency_F','Monetary_M']]

,고객ID,Recency_R,Frequency_F,Monetary_M
0,A0063H17IS0OELIO,3,1,1
1,A007E35OUXTJ3BRR,5,1,1
2,A008UGCQHSONWYDJ,3,1,1
3,A00AXO66OKGQAC8F,5,1,1
4,A00BD03LKJIH6LTU,2,4,5
...,...,...,...,...
32487,AZZY0IZN75VOACLI,3,1,1
32488,AZZYI8XJU7OCXO73,1,1,1
32489,AZZYZ8SXW66M0RBL,5,4,3
32490,AZZZ70FWYOY16G62,1,5,5


### ***rfm_score data 설명***

1. Recency = 최근 구매일
2. Frequency = 방문횟수
3. Monetary = 구매횟수
4. Recency_R = 최근 구매일 점수, 1~5점으로 점수가 높을수록 좋다.
5. Frequency_F = 방문횟수 점수, 1~5점으로 점수가 높을수록 좋다.
6. Monetary_M = 구매횟수 점수, 1~5점으로 점수가 높을수록 좋다.

In [ ]:
## data load
rfm_score = pd.read_csv('result.csv')
rfm_score.head(10)

,고객ID,Recency,Frequency,Monetary,Recency_R,Frequency_F,Monetary_M
0,A0063H17IS0OELIO,2018-04-06,1,1,3,1,1
1,A007E35OUXTJ3BRR,2021-06-22,1,1,5,1,1
2,A008UGCQHSONWYDJ,2019-09-25,1,1,3,1,1
3,A00AXO66OKGQAC8F,2021-09-14,1,1,5,1,1
4,A00BD03LKJIH6LTU,2016-11-02,2,4,2,4,5
5,A00BKTGNMZXXU9NB,2015-09-25,1,1,2,1,1
6,A00E949YXDIP24UM,2021-12-30,1,1,5,1,1
7,A00EIFH5MF1D3N68,2010-04-19,1,3,1,1,4
8,A00EKMHO59OQNRCX,2016-12-16,1,2,2,1,3
9,A00FF7U0QU7LK2E5,2012-02-20,2,2,1,4,3


In [ ]:
rfm_score = rfm_score[['고객ID', 'Monetary', 'Recency_R','Frequency_F','Monetary_M']]

In [ ]:
def get_score(level, data, reverse = False):
    score = [] 
    for j in range(len(data)): 
        for i in range(len(level)): 
            if data[j] <= level[i]: 
                score.append(i+1) 
                break 
            elif data[j] > max(level): 
                score.append(len(level)+1) 
                break 
            else: 
                continue
    if reverse:
        return [len(level)+2-x for x in score]
    else:
        return score 
 
grid_number = 100 ## 눈금 개수, 너무 크게 잡으면 메모리 문제가 발생할 수 있음.
weights = []
for j in range(grid_number+1):
    weights += [(i/grid_number,j/grid_number,(grid_number-i-j)/grid_number)
                  for i in range(grid_number+1-j)]
num_class = 5 ## 클래스 개수
class_level = np.linspace(1,5,num_class+1)[1:-1] ## 클래스를 나누는 지점을 정한다.
total_amount_of_sales = rfm_score['Monetary'].sum() ## 구매금액 총합 = 총 매출

In [ ]:
# class의 구간치, 1.8 이하면 5등급, 4.2이상이면 1등급 고객

num_class = 5
class_level = np.linspace(1,5,num_class+1)[1:-1] ## 클래스를 나누는 지점을 정한다.
print(class_level)

[1.8 2.6 3.4 4.2]


In [ ]:
## 최적의 표준편차 값을 구하는 Logic

max_std = 0 ## 표준편차 초기값

for w in tqdm(weights,position=0,desc = '[Finding Optimal weights]'):
    ## 주어진 가중치에 따른 고객별 점수 계산
    score = w[0]*rfm_score['Recency_R'] + \
                        w[1]*rfm_score['Frequency_F'] + \
                        w[2]*rfm_score['Monetary_M'] 
    rfm_score['Class'] = get_score(class_level,score,True) ## 점수를 이용하여 고객별 등급 부여
    ## 등급별로 구매금액을 집계한다.
    grouped_rfm_score = rfm_score.groupby('Class')['Monetary'].sum().reset_index()

    ## 등급이 높은 고객들의 매출이 낮은 등급의 고객들보다 커야한다.
    grouped_rfm_score = grouped_rfm_score.sort_values('Class')
    temp_monetary = list(grouped_rfm_score['Monetary'])
    if temp_monetary != sorted(temp_monetary,reverse=True):
        continue
        
    ## 클래스별 구매금액을 총구매금액으로 나누어 클래스별 매출 기여도 계산
    grouped_rfm_score['Monetary'] = grouped_rfm_score['Monetary'].map(lambda x : x/total_amount_of_sales)
    std_sales = grouped_rfm_score['Monetary'].std() ## 매출 기여도의 표준편차 계산

    ## 표준편차 최대값 업데이트 및 가중치 업데이트
    if max_std <= std_sales:
        max_std = std_sales 
        optimal_weights = w

[Finding Optimal weights]:   4%|▎         | 192/5151 [01:07<28:51,  2.86it/s]


KeyboardInterrupt: 

In [ ]:
print('Recency_R, Frequency_F, Monetary_M의 최적의 가중치 :',optimal_weights)

Recency_R, Frequency_F, Monetary_M의 최적의 가중치 : (0.36, 0.04, 0.6)


In [ ]:
score = optimal_weights[0]*rfm_score['Recency_R'] + \
        optimal_weights[1]*rfm_score['Frequency_F'] + \
        optimal_weights[2]*rfm_score['Monetary_M'] ## 고객별 점수 계산

rfm_score['Class'] = get_score(class_level,score,True) ## 고객별 등급 부여

In [ ]:
rfm_score

,고객ID,Monetary,Recency_R,Frequency_F,Monetary_M,Class
0,A0063H17IS0OELIO,1,3,1,1,5
1,A007E35OUXTJ3BRR,1,5,1,1,4
2,A008UGCQHSONWYDJ,1,3,1,1,5
3,A00AXO66OKGQAC8F,1,5,1,1,4
4,A00BD03LKJIH6LTU,4,2,4,5,2
...,...,...,...,...,...,...
32487,AZZY0IZN75VOACLI,1,3,1,1,5
32488,AZZYI8XJU7OCXO73,1,1,1,1,5
32489,AZZYZ8SXW66M0RBL,2,5,4,3,2
32490,AZZZ70FWYOY16G62,6,1,5,5,2


In [ ]:
## 클래스별 고객 수 계산
temp_rfm_score1 = rfm_score.groupby('Class')['고객ID'].count().reset_index().rename(columns={'CustomerID':'Count'})
 
## 클래스별 구매금액(매출)계산
temp_rfm_score2 = rfm_score.groupby('Class')['Monetary'].sum().reset_index()

## 클래스별 매출 기여도 계산
temp_rfm_score2['Contribution'] = temp_rfm_score2['Monetary'].map(lambda x : x/total_amount_of_sales)
 
## 데이터 결합
result_df = pd.merge(temp_rfm_score1,temp_rfm_score2,how='left',on=('Class'))

In [ ]:
result_df

,Class,고객ID,Monetary
0,1,3545,0.347959
1,2,3894,0.186004
2,3,5488,0.173192
3,4,9184,0.157685
4,5,10381,0.135160
